In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
import time
from datetime import datetime
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
SEED = 10

In [3]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution started at 2022-02-23 02:04:05.


In [4]:
filepath = "../Data Preprocessing/iot23_combined_1000k.csv"
df = pd.read_csv(filepath)

In [5]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,id.resp_h_99.99.124.201,id.resp_p_1,id.resp_p_23,id.resp_p_80,id.resp_p_123,id.resp_p_2323,id.resp_p_8080,id.resp_p_8081,id.resp_p_9527,id.resp_p_62336
0,65,2.999062,0,0,0,3,180,0,0,PartOfAHorizontalPortScan,...,0,0,0,0,0,1,0,0,0,0
1,70,2.999066,0,0,0,3,180,0,0,PartOfAHorizontalPortScan,...,0,0,1,0,0,0,0,0,0,0
2,73,2.999056,0,0,0,3,180,0,0,PartOfAHorizontalPortScan,...,0,0,1,0,0,0,0,0,0,0
3,75,0.000000,0,0,0,1,60,0,0,PartOfAHorizontalPortScan,...,0,0,0,0,0,1,0,0,0,0
4,79,0.000000,0,0,0,1,60,0,0,PartOfAHorizontalPortScan,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65573,393,0.000000,0,0,0,1,76,0,0,Benign,...,0,0,0,0,1,0,0,0,0,0
65574,210294,0.000000,0,0,0,1,68,0,0,Benign,...,0,1,0,0,0,0,0,0,0,0
65575,257557,0.000000,0,0,0,1,68,0,0,Benign,...,0,1,0,0,0,0,0,0,0,0
65576,273601,0.000000,0,0,0,1,68,0,0,Benign,...,0,1,0,0,0,0,0,0,0,0


In [6]:
del df['Unnamed: 0']

In [7]:
df['label'].value_counts()

DDoS                         25458
Benign                       23383
PartOfAHorizontalPortScan    16728
C&C                              7
C&C-FileDownload                 2
Name: label, dtype: int64

In [8]:
good = 'Benign'
bad = 'PartOfAHorizontalPortScan'
filtered_labels = df['label'].value_counts().index.drop([good,bad])
for label in filtered_labels:
    df.drop(df[df.label == label].index, inplace=True)

In [9]:
df['label'].value_counts()

Benign                       23383
PartOfAHorizontalPortScan    16728
Name: label, dtype: int64

In [10]:
df.loc[(df.label == good), 'label'] = 0
df.loc[(df.label == bad), 'label'] = 1
df = df.astype({'label': int})

In [11]:
df['label'].value_counts()

0    23383
1    16728
Name: label, dtype: int64

In [12]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,id.resp_h_99.99.124.201,id.resp_p_1,id.resp_p_23,id.resp_p_80,id.resp_p_123,id.resp_p_2323,id.resp_p_8080,id.resp_p_8081,id.resp_p_9527,id.resp_p_62336
0,2.999062,0,0,0,3,180,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,2.999066,0,0,0,3,180,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,2.999056,0,0,0,3,180,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,0.000000,0,0,0,1,60,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,0.000000,0,0,0,1,60,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65573,0.000000,0,0,0,1,76,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
65574,0.000000,0,0,0,1,68,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
65575,0.000000,0,0,0,1,68,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
65576,0.000000,0,0,0,1,68,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [13]:
X = df.loc[:, df.columns != 'label']
print(X.shape)

(40111, 6845)


In [14]:
Y = df['label'].values
print(Y.shape)

(40111,)


In [15]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFpr, SelectFdr, SelectFwe
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
X = MinMaxScaler(feature_range=(0,1)).fit_transform(X,Y)
X = SelectPercentile(percentile=1).fit_transform(X,Y)
print(X.shape)

C:\Users\marce\Anaconda2\envs\doutorado\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   3   44   46   48   49   51   53   54   57   58   59   60   61  676
  832 1387 2548 5286 5986 6844] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\marce\Anaconda2\envs\doutorado\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


(40111, 69)


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=SEED, test_size=0.2)

In [17]:
model = ak.StructuredDataClassifier(overwrite=True, max_trials=1)

In [18]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, validation_split=0.25)
print(history)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

Trial 1 Complete [00h 00m 48s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 48s
INFO:tensorflow:Oracle triggered exit
1003/1003 [==============================] - 2s 985us/step - loss: 0.0397 - accuracy: 0.9902
INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets

program end...

time cost: 
94.17593955993652 seconds


In [19]:
best_model = model.export_model()

In [20]:
print("Evaluate on test data")
results = best_model.evaluate(X_test, Y_test)
print("test loss, test acc:", results)

Evaluate on test data
251/251 [==============================] - 1s 1ms/step - loss: 0.0204 - accuracy: 0.9976
test loss, test acc: [0.020351147279143333, 0.9976317882537842]


In [21]:
print(f"Execution finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution finished at 2022-02-23 02:06:36.
